In [1]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split

# Paths
fact_table_path = "images/training/fact_table.csv"
segments_dir = "images/training/segments"
output_dir = "dataset"

# Create output directories
train_images_dir = os.path.join(output_dir, "train/images")
val_images_dir = os.path.join(output_dir, "val/images")
os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(val_images_dir, exist_ok=True)

# Load the fact_table CSV
fact_table = pd.read_csv(fact_table_path)

# Filter out rows with "__OMIT__" in the true_label column
fact_table = fact_table[fact_table["true_label"] != "__OMIT__"]

# Split the dataset into 80% train and 20% validation
train_df, val_df = train_test_split(fact_table, test_size=0.2, random_state=42)

# Function to copy images and write labels.txt
def process_split(df, images_dir, labels_file):
    with open(labels_file, "w") as f:
        for _, row in df.iterrows():
            src_image_path = os.path.join(segments_dir, row["segment_file_path"])
            dest_image_path = os.path.join(images_dir, row["segment_file_path"])
            
            # Copy the image to the destination directory
            shutil.copy(src_image_path, dest_image_path)
            
            # Write the label to labels.txt
            f.write(f"{row['segment_file_path']}\t{row['true_label']}\n")

# Process the training split
process_split(train_df, train_images_dir, os.path.join(output_dir, "train/labels.txt"))

# Process the validation split
process_split(val_df, val_images_dir, os.path.join(output_dir, "val/labels.txt"))

print("Dataset successfully reorganized for EasyOCR fine-tuning!")

Dataset successfully reorganized for EasyOCR fine-tuning!


In [ ]:
import os
import yaml
from easyocr.config import get_config
from easyocr.utils import download_and_unzip
from easyocr.train import train

# Paths
dataset_dir = "dataset"
train_labels = os.path.join(dataset_dir, "train/labels.txt")
val_labels = os.path.join(dataset_dir, "val/labels.txt")
train_images_dir = os.path.join(dataset_dir, "train/images")
val_images_dir = os.path.join(dataset_dir, "val/images")
output_dir = "output"

# Create output directory
os.makedirs(output_dir, exist_ok=True)

# Configuration for fine-tuning
config = {
    "model": {
        "name": "custom_model",
        "pretrained": True,  # Use pretrained weights as a starting point
        "num_classes": 37,  # Adjust this to the number of unique characters in your dataset
    },
    "dataset": {
        "train": train_labels,
        "val": val_labels,
        "root": dataset_dir,
    },
    "training": {
        "batch_size": 16,
        "epochs": 50,
        "learning_rate": 0.001,
        "optimizer": "adam",
        "scheduler": "step",
        "step_size": 10,
        "gamma": 0.1,
    },
    "output": {
        "dir": output_dir,
    },
}

# Save the configuration to a YAML file
config_path = os.path.join(output_dir, "training_config.yaml")
with open(config_path, "w") as f:
    yaml.dump(config, f)

# Download pretrained weights if needed
pretrained_weights_url = "https://github.com/JaidedAI/EasyOCR/releases/download/v1.3/english_g2.pth"
pretrained_weights_path = os.path.join(output_dir, "pretrained.pth")
if not os.path.exists(pretrained_weights_path):
    print("Downloading pretrained weights...")
    download_and_unzip(pretrained_weights_url, output_dir)
 
# Load the configuration
cfg = get_config(config_path)

# Fine-tune the model
print("Starting fine-tuning...")
train(cfg)
print("Fine-tuning complete!")

ImportError: cannot import name 'get_config' from 'easyocr.config' (/Users/jae/Documents/Personal projects/SMT inventory OCR/smt-inventory-ocr/.venv/lib/python3.10/site-packages/easyocr/config.py)